In [1]:
import pandas as pd
import pandas as pd
import numpy as np
import ast
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import re

In [5]:
df = pd.read_csv('/Users/com/Documents/GitHub/advanced_project/jiwon_project/csv_files/NY_Airbnb_original_df.csv')

In [6]:
# 전처리 1 - id int로 변경
df['id'] = df['id'].astype(int)

In [ ]:
df.info()

In [ ]:
# booking_info
df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
df['instant_bookable'] =  df['instant_bookable'].map({'f':0, 't':1}) #instant_bookable
df['is_long_term'] = (df['minimum_nights'] >= 28).astype(int)

<>:2: SyntaxWarning: invalid escape sequence '\$'
<>:2: SyntaxWarning: invalid escape sequence '\$'
C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_8904\2729345397.py:2: SyntaxWarning: invalid escape sequence '\$'
  df['price'] = df['price'].replace('[\$,]', '', regex=True).astype(float)
